# Data preparation

In this notebook we will build a parquet file for use in further analyses. 

## Imports

In [1]:
import os, zipfile, itables
import pandas as pd
itables.options.style = 'float:left;width:auto'

## Load Data

In [2]:
files = os.listdir('data')
files = [file for file in files if file.endswith('.zip')]

In [3]:
files

['bdc_53_Cable_fixed_broadband_123122-20230926.zip',
 'bdc_53_Copper_fixed_broadband_123122-20230926.zip',
 'bdc_53_FibertothePremises_fixed_broadband_123122-20230926.zip',
 'bdc_53_GSOSatellite_fixed_broadband_123122-20230926.zip',
 'bdc_53_LBRFixedWireless_fixed_broadband_123122-20230926.zip',
 'bdc_53_LicensedFixedWireless_fixed_broadband_123122-20230926.zip',
 'bdc_53_NGSOSatellite_fixed_broadband_123122-20230926.zip',
 'bdc_53_UnlicensedFixedWireless_fixed_broadband_123122-20230926.zip']

In [4]:
techHash = {
    'technology': pd.Series([0, 10, 40, 50, 60, 61, 70, 71, 72]),
    'techName': pd.Series(['Other', 'Copper', 'Cable', 'Fiber', 'GSO', 'NGSO', 'UFW', 'LFW', 'LBRW'])
}
techDf = pd.DataFrame(techHash)

In [5]:
waAvailabilityData = pd.DataFrame()
dfColumnHints = {
    'block_geoid': str
}

for file in files:
    # print(file)
    archive = zipfile.ZipFile(os.path.join('data', file))
    with archive.open(archive.filelist[0].filename) as f:
        localDf = pd.read_csv(f, dtype=dfColumnHints)
    waAvailabilityData = pd.concat([waAvailabilityData, localDf], ignore_index=True)

waAvailabilityData = waAvailabilityData.merge(techDf, how='left')

wsboUnserved = pd.read_csv(os.path.join('data', 'unserved.csv'))
wsboUnderserved = pd.read_csv(os.path.join('data', 'underserved.csv'))

In [6]:
print(f'WSBO Unserved: {wsboUnserved.shape[0]}')
print(f'WSBO Underserved: {wsboUnderserved.shape[0]}')

WSBO Unserved: 236134
WSBO Underserved: 85434


### Parse out Locations

In [7]:
waLocations = waAvailabilityData[['location_id', 'block_geoid', 'h3_res8_id']]

In [8]:
waLocations = waLocations.drop_duplicates()
waLocations['county_geoid'] = waLocations.block_geoid.str[0:5]

In [9]:
waLocations

,location_id,block_geoid,h3_res8_id,county_geoid
0,1015880838,530659514022053,8812db0811fffff,53065
1,1073176227,530350910012001,8828d50b39fffff,53035
2,1073185639,530350907004015,8828d546bdfffff,53035
3,1056655277,530499502005010,8828c6c015fffff,53049
4,1073186253,530350909022006,8828d554b7fffff,53035
...,...,...,...,...
12535029,1310087661,530730002032000,8828d17801fffff,53073
12535061,1049483490,530479709001124,8828d2ab6bfffff,53047
12535205,1310144322,530730103011029,8828d169edfffff,53073
12535621,1312148589,530050118021061,882889d269fffff,53005


## WSBO Service

In [10]:
waLocations['wsbo'] = 'Served'

In [11]:
waLocations.loc[waLocations.location_id.isin(wsboUnderserved.location_id), 'wsbo'] = 'Underserved'
waLocations.loc[waLocations.location_id.isin(wsboUnserved.location_id), 'wsbo'] = 'Unserved'

In [12]:
waLocations.wsbo.value_counts()

wsbo
Served         2206330
Unserved        236134
Underserved      85434
Name: count, dtype: int64

## FCC Technology Service 

In [13]:
itables.show(waAvailabilityData.technology.value_counts().reset_index().merge(techDf, how='left'))

### Cable Service Categories

In [14]:
waLocations['cableServed'] = 'Unserved'
techData = waAvailabilityData[waAvailabilityData.technology == 40]
techLocations = techData[((techData.max_advertised_download_speed >= 25) & (techData.max_advertised_upload_speed >= 3))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'cableServed'] = 'Underserved'
techLocations = techData[((techData.max_advertised_download_speed >= 100) & (techData.max_advertised_upload_speed >= 20))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'cableServed'] = 'Served'

### Copper Service Categories

In [15]:
waLocations['copperServed'] = 'Unserved'
techData = waAvailabilityData[waAvailabilityData.technology == 10]
techLocations = techData[((techData.max_advertised_download_speed >= 25) & (techData.max_advertised_upload_speed >= 3))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'copperServed'] = 'Underserved'
techLocations = techData[((techData.max_advertised_download_speed >= 100) & (techData.max_advertised_upload_speed >= 20))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'copperServed'] = 'Served'

### Fiber Service Categories

In [16]:
waLocations['fiberServed'] = 'Unserved'
techData = waAvailabilityData[waAvailabilityData.technology == 50]
techLocations = techData[((techData.max_advertised_download_speed >= 25) & (techData.max_advertised_upload_speed >= 3))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'fiberServed'] = 'Underserved'
techLocations = techData[((techData.max_advertised_download_speed >= 100) & (techData.max_advertised_upload_speed >= 20))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'fiberServed'] = 'Served'

### Geo-Stationary Orbit

In [17]:
waLocations['gsoServed'] = 'Unserved'
techData = waAvailabilityData[waAvailabilityData.technology == 60]
techLocations = techData[((techData.max_advertised_download_speed >= 25) & (techData.max_advertised_upload_speed >= 3))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'gsoServed'] = 'Underserved'
techLocations = techData[((techData.max_advertised_download_speed >= 100) & (techData.max_advertised_upload_speed >= 20))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'gsoServed'] = 'Served'

### LBR Wireless

In [18]:
waLocations['lbrwServed'] = 'Unserved'
techData = waAvailabilityData[waAvailabilityData.technology == 72]
techLocations = techData[((techData.max_advertised_download_speed >= 25) & (techData.max_advertised_upload_speed >= 3))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'lbrwServed'] = 'Underserved'
techLocations = techData[((techData.max_advertised_download_speed >= 100) & (techData.max_advertised_upload_speed >= 20))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'lbrwServed'] = 'Served'

### Licensed Fixed Wireless

In [19]:
waLocations['lfwServed'] = 'Unserved'
techData = waAvailabilityData[waAvailabilityData.technology == 71]
techLocations = techData[((techData.max_advertised_download_speed >= 25) & (techData.max_advertised_upload_speed >= 3))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'lfwServed'] = 'Underserved'
techLocations = techData[((techData.max_advertised_download_speed >= 100) & (techData.max_advertised_upload_speed >= 20))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'lfwServed'] = 'Served'

### Non-Geostationary Orbit

In [20]:
waLocations['ngsoServed'] = 'Unserved'
techData = waAvailabilityData[waAvailabilityData.technology == 61]
techLocations = techData[((techData.max_advertised_download_speed >= 25) & (techData.max_advertised_upload_speed >= 3))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'ngsoServed'] = 'Underserved'
techLocations = techData[((techData.max_advertised_download_speed >= 100) & (techData.max_advertised_upload_speed >= 20))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'ngsoServed'] = 'Served'

### Unlicensed Fixed Wireless

In [21]:
waLocations['ufwServed'] = 'Unserved'
techData = waAvailabilityData[waAvailabilityData.technology == 70]
techLocations = techData[((techData.max_advertised_download_speed >= 25) & (techData.max_advertised_upload_speed >= 3))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'ufwServed'] = 'Underserved'
techLocations = techData[((techData.max_advertised_download_speed >= 100) & (techData.max_advertised_upload_speed >= 20))].location_id.reset_index(drop=True)
waLocations.loc[waLocations.location_id.isin(techLocations), 'ufwServed'] = 'Served'

In [22]:
itables.show(waLocations, n=15)

## Save the data!

In [23]:
waLocations.to_parquet(os.path.join('data', 'parsedData.parquet'))

## Under/Unserved

In [24]:
print(f'WSBO Unserved: {wsboUnserved.shape[0]}')

WSBO Unserved: 236134


In [25]:
servedStats = (waLocations['cableServed'].astype('str') 
    + ',' + waLocations['copperServed'].astype('str')
    + ',' + waLocations['fiberServed'].astype('str')
    + ',' + waLocations['lbrwServed'].astype('str')
    + ',' + waLocations['lfwServed'].astype('str')).value_counts().reset_index().rename(columns = {'index': 'status'})

In [26]:
# itables.show(servedStats)
servedStats[servedStats.status == 'Unserved,Unserved,Unserved,Unserved,Unserved']

,status,count
3,"Unserved,Unserved,Unserved,Unserved,Unserved",232495


### Remove LBR Wireless

In [27]:
servedStats = (waLocations['cableServed'].astype('str') 
    + ',' + waLocations['copperServed'].astype('str')
    + ',' + waLocations['fiberServed'].astype('str')
    + ',' + waLocations['lfwServed'].astype('str')).value_counts().reset_index().rename(columns = {'index': 'status'})

In [28]:
servedStats[servedStats.status == 'Unserved,Unserved,Unserved,Unserved']

,status,count
3,"Unserved,Unserved,Unserved,Unserved",239564


# Unused Stuff

In [29]:
unservedWaLocations = waLocations.merge(wsboUnserved, how='outer', indicator=True)

In [30]:
unservedWaLocations = unservedWaLocations[unservedWaLocations._merge != 'left_only']

In [31]:
# waLocations.loc[waLocations.location_id == 1015880838, 'fiberserved'] = 'foo'

In [32]:
waLocations.fiberserved.value_counts()

AttributeError: 'DataFrame' object has no attribute 'fiberserved'

In [ ]:
waFiberData

In [ ]:
waLocations.cableServed.value_counts()